In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import spacy
import tools
import imp
from sklearn.metrics import make_scorer
import time
from sklearn.decomposition import online_lda
from sklearn.linear_model import LogisticRegression


In [2]:
imp.reload(tools)
label_encoder,labels = tools.read_labels()
total_vec,myCViterator = tools.total_vect_cv_prepare()
default_vectorizer,no_filtered_vec = tools.NB_data_prepare()
sw = tools.stop_word()
lemma_text = tools.read_lemma_data()

74260
74260


# Find best set of hyper parameters & get a Linear Regression model
---

### find best hyper parameters:


In [12]:
from sklearn.metrics import make_scorer
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from pipeline_helper import PipelineHelper
import tools
sw = tools.stop_word()
lemma_text = tools.read_lemma_data()
X = lemma_text
Y = labels

# A grid search pipeline that includes all combination of vectorizer and classifier
pipe = Pipeline([
    ('vectorizer', PipelineHelper([
        ('count', CountVectorizer(stop_words=sw)),
        ('tfidf', TfidfVectorizer(stop_words=sw)),
    ])),
    ('classifier', PipelineHelper([
        ('LR_clf', LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial')),
    ])),
])
    
params = {
    'vectorizer__selected_model': pipe.named_steps['vectorizer'].generate({
        'count__ngram_range': [(1,1)],
        'count__min_df': [5,10],
        'count__max_df': [0.25,0.5,0.75],
        'count__token_pattern': [r"(?u)\b\d?[a-z]{2,}\d?\b",r"(?u)\b[a-z]{2,}\b"],
        'tfidf__ngram_range': [(1,1)],
        'tfidf__min_df':[5,10],
        'tfidf__max_df':[0.25,0.5,0.75],
        'tfidf__token_pattern': [r"(?u)\b\d?[a-z]{2,}\d?\b",r"(?u)\b[a-z]{2,}\b"]
    }),
    'classifier__selected_model': pipe.named_steps['classifier'].generate({
        'LR_clf__fit_intercept': [True,False],
        'LR_clf__class_weight': [None,'balanced'],
        'LR_clf__C': [1,3],
    })
}
grid = GridSearchCV(pipe, params, scoring='accuracy', verbose=1,cv=5,n_jobs=38)
grid.fit(X, Y)
print(grid.best_params_)
print(grid.best_score_)

Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=38)]: Using backend LokyBackend with 38 concurrent workers.
[Parallel(n_jobs=38)]: Done 124 tasks      | elapsed: 12.7min
[Parallel(n_jobs=38)]: Done 374 tasks      | elapsed: 31.4min
[Parallel(n_jobs=38)]: Done 724 tasks      | elapsed: 61.8min
[Parallel(n_jobs=38)]: Done 960 out of 960 | elapsed: 80.2min finished


{'vectorizer__selected_model': ('tfidf', {'max_df': 0.5, 'token_pattern': '(?u)\\b\\d?[a-z]{2,}\\d?\\b', 'min_df': 10, 'ngram_range': (1, 1)}), 'classifier__selected_model': ('LR_clf', {'C': 1, 'fit_intercept': True, 'class_weight': None})}
0.8615001346619984


/export/home/tzzhang/.pyenv/versions/3.5.2/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


### reproduce the best classifier to verifier the result of grid search

In [35]:
Tvectorizer = TfidfVectorizer(max_df = 0.5,min_df=10,stop_words=sw,token_pattern=r'\b\d?[a-z]{2,}\d?\b',ngram_range=(1,1))
Tfidf_vec = Tvectorizer.fit_transform(lemma_text)

In [36]:
np.shape(Tfidf_vec)

(74260, 8116)

In [42]:
X_train, X_test, y_train, y_test = train_test_split(Tfidf_vec, labels, test_size=0.2, random_state=40)
best_clf = LogisticRegression(random_state=0, solver='lbfgs',fit_intercept =False,multi_class='multinomial').fit(X_train, y_train)
y_pred = best_clf.predict(X_test)
print(metrics.classification_report(y_test, y_pred, target_names=label_encoder.classes_))

                   precision    recall  f1-score   support

          Access        0.71      0.55      0.62      2089
Delivery of care        0.88      0.94      0.91     10565
     Environment        0.80      0.69      0.74       615
         Finance        0.89      0.84      0.86      1583

         accuracy                           0.86     14852
        macro avg       0.82      0.75      0.78     14852
     weighted avg       0.86      0.86      0.86     14852



### When training the final classifier, all data was used for training.

In [43]:
final_clf = LogisticRegression(random_state=0, solver='lbfgs',fit_intercept =False,multi_class='multinomial').fit(Tfidf_vec, labels)

### write keywords into dummy documents

In [ ]:
imp.reload(tools)
dummy_list,words_indexes_list = tools.show_word_linear(Cvectorizer,count_vec_per_class,final_clf,label_encoder,des_file = 'dummy_doc_1015_tfidf_exclusive',exclusive=True,norm=True)

# Get the best LDA model by custom loss
---

In [4]:
sw = tools.stop_word()
FIXED_SEED = 0
lemma_text = tools.read_lemma_data()
imp.reload(tools)
Cvectorizer = CountVectorizer(max_df = 0.5,min_df=10,stop_words=sw,token_pattern=r"(?u)\b\d?[a-z]{2,}\d?\b",ngram_range=(1,1))
Cvectorizer.fit_transform(lemma_text)
total_vec,myCViterator = tools.total_vect_cv_prepare(vectorizer=Cvectorizer,dummy_document = 'dummy_doc_1015_tfidf_exclusive')
Count_vec = total_vec[:-4]

In [4]:
np.shape(Count_vec)

(73093, 8116)

In [5]:
from sklearn.metrics import make_scorer
my_scorer = make_scorer(tools.NB_loss_function)
print(my_scorer)

make_scorer(NB_loss_function)


In [28]:
from sklearn.decomposition import online_lda
import sklearn
imp.reload(sklearn)
imp.reload(tools)
#max_doc_update_iter=50,learning_decay
parameters = {'batch_size':[128,256,512],'learning_offset':[256,64,1024],
              'n_components':[40],'learning_decay':[0.8,0.9]}

lda_for_gs = online_lda.LatentDirichletAllocation(random_state=40,\
                                           learning_method='online',max_iter=10,n_jobs=1,verbose=1)
stop_word_list = []
X = total_vec
dummy_label = np.array([0,0,0,0])
Y=np.hstack([labels,dummy_label])
Y = Y[:X.shape[0]]
#scoring = my_score
# before training/inference:
grid = sklearn.model_selection.GridSearchCV(lda_for_gs,param_grid=parameters,verbose=1,cv=myCViterator,scoring = my_scorer,n_jobs=30,refit=True)
grid.fit(X,Y)
print(grid.best_params_)
print(grid.best_score_)

Fitting 1 folds for each of 18 candidates, totalling 18 fits


[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done  16 out of  18 | elapsed: 31.7min remaining:  4.0min
[Parallel(n_jobs=30)]: Done  18 out of  18 | elapsed: 31.9min finished


iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10
{'learning_decay': 0.9, 'batch_size': 256, 'learning_offset': 256, 'n_components': 40}
-8.078364470258512


Gridsearch will re-train a model according to this hyper-parameter after finding the optimal hyper-parameter, and the loss of the newly trained model will not be exactly equal to the previous model.

In [131]:
NB_vect = total_vec[-4:]
lda = grid.best_estimator_
NB_topic_dist = lda.transform(NB_vect)
#report_topic = lda2.transform(count_vec)
print('Access : ')
to_topic = np.argpartition(NB_topic_dist[0],-5)[-5:]
print(to_topic)
print(NB_topic_dist[0,to_topic])

print('\nDelivery of care : ')
to_topic = np.argpartition(NB_topic_dist[1],-5)[-5:]
print(to_topic)
print(NB_topic_dist[1,to_topic])

print('\nEnvironment : ')
to_topic = np.argpartition(NB_topic_dist[2],-5)[-5:]
print(to_topic)
print(NB_topic_dist[2,to_topic])

print('\nFinance : ')
to_topic = np.argpartition(NB_topic_dist[3],-5)[-5:]
print(to_topic)
print(NB_topic_dist[3,to_topic])

tools.NB_loss_function(0,NB_topic_dist)

Access : 
[26 10  6 11 21]
[0.06623853 0.06766384 0.0769857  0.35946146 0.24938432]

Delivery of care : 
[33 21 26 34  6]
[0.058395   0.1089188  0.36508012 0.07440465 0.0752569 ]

Environment : 
[19 34 37 21 15]
[0.00965216 0.10242343 0.56212395 0.03005608 0.27345419]

Finance : 
[21 33 22 27 19]
[0.05992373 0.06328937 0.10240512 0.32412602 0.41950788]
Entropy Loss: -7.0112849185214605
Corr Loss: 0.15202521266924354


-8.531537045213895

In [129]:
import imp
imp.reload(tools)
Count_vec = total_vec[:-4]
topics = lda.transform(Count_vec)
topic_count = np.sum(topics,axis=0)
topic_sort = np.argsort(topic_count)
rev_sort = []
for i in range(len(topic_sort)-1,-1,-1):
    rev_sort.append(topic_sort[i])
representative_words = tools.print_topic_word(lda,Cvectorizer,Count_vec,word_num=40,weight_list = rev_sort)

Topic 26 appointment, wait, time, clinic, speak, book, clerk, information, contact, explain, phone, office, number, staff, hour, receptionist, doctor, process, schedule, ask, person, leave, procedure, like, cancel, follow, question, appt, date, answer, pcc, advise, booking, happen, request, year, minute, provide, day,

Topic 22 care, home, staff, facility, provide, manager, nursing, plan, term, change, case, meeting, dementia, homecare, transition, bath, need, palliative, senior, hc, management, lack, long, support, ltc, caregiver, agent, living, receive, transfer, client, assist, assistance, level, ensure, staffing, live, resident, lodge,

Topic 33 nurse, come, triage, ask, leave, staff, room, sit, rude, iv, way, hand, check, comment, talk, unprofessional, shift, desk, nursing, clean, hour, night, treat, manner, look, walk, yell, minute, help, attitude, like, reply, start, wait, male, cry, water, pain, compassion,

Topic 32 ed, physician, pain, ct, scan, rdrhc, symptom, emergency, dia

In [130]:
topic_count = np.sum(topics,axis=0)
topic_count = topic_count / np.sum(topic_count)
log = 'Topic {}: {:0.4f} '
for i in np.argsort(-topic_count):
    print(log.format(i,topic_count[i]))

Topic 26: 0.1058 
Topic 22: 0.0882 
Topic 33: 0.0880 
Topic 32: 0.0752 
Topic 21: 0.0720 
Topic 9: 0.0492 
Topic 15: 0.0485 
Topic 1: 0.0427 
Topic 34: 0.0408 
Topic 27: 0.0388 
Topic 11: 0.0358 
Topic 16: 0.0352 
Topic 17: 0.0338 
Topic 18: 0.0314 
Topic 29: 0.0311 
Topic 19: 0.0307 
Topic 31: 0.0274 
Topic 25: 0.0211 
Topic 30: 0.0156 
Topic 24: 0.0105 
Topic 28: 0.0096 
Topic 36: 0.0076 
Topic 2: 0.0059 
Topic 39: 0.0054 
Topic 8: 0.0045 
Topic 35: 0.0044 
Topic 7: 0.0042 
Topic 3: 0.0041 
Topic 20: 0.0037 
Topic 37: 0.0036 
Topic 6: 0.0034 
Topic 10: 0.0033 
Topic 13: 0.0030 
Topic 12: 0.0029 
Topic 14: 0.0026 
Topic 0: 0.0023 
Topic 23: 0.0022 
Topic 38: 0.0020 
Topic 5: 0.0018 
Topic 4: 0.0017 
